# Parsowanie plików
Procedura parsowania wygląda następująco
1. Patrzymy na plik z symulacji/pomiaru, sprawdzamy którą kolumnę i wiersze mamy wczytać
2. Następnie podajemy argumenty klasy processFile:
    a) arrname - nazwa tablicy w której będą dane
    b) filename  - nazwa pliku który otworzymy
    c) start - pierwszy wiersz jaki będzie wczytany
    d) end - ostatni wiersz jaki będzie wczytany
    e) col - numer kolumny która będzie wczytana
    f) dgb - flaga do debugowania (nie używać)
    g) magic_time - wyniki pomiarów z oscyloskopu mają nietypowy format np. wyglądają tak "17:53:58,785136", jeśli magic_time jest true pierwsza część jest ucinana a druga parsowana (ostatecznie otrzymamy tutaj "785136")
3. Kopiujemy tak uzyskane wyjścid do pola z kodem

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import matplotlib.ticker as ticker

def printArray(name,array):
    line = name
    line +=str("=np.array([")
    size = array.size
    for i in range(0, size-1):
        line +=str(str(array[i])+",")
    line += str(str(array[size-1])+"])")
    print(line)
    
def processFile(arrname, filename, start=0, end=10, col=0, dbg=False, magic_time=False, step=1):
    lineNo = 0
    res = []
    with open(filename) as infile:
        for line in infile:
            lineNo+=1
            if(lineNo>=start and lineNo<=end):
                array = line.split()
                if lineNo%step !=0:
                    continue
                if dbg:
                    print(array[col])
                try:
                    if magic_time:
                        temp = array[col]
                        temp = temp.split(',')
                        res.append(int(temp[1]))
                    else:
                        array[col] = array[col].replace(',',".")
                        res.append(float(array[col]))
                except Exception as error:
                    print(error)
                    pass
    
    nparr = np.array(res)
    if dbg:
        print(nparr)
        print("size ",nparr.size)
    return printArray(arrname, np.array(res))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import matplotlib.ticker as ticker

def makePhasePlotSim(freq, gain, phase, freqHz=0., ):
    G = gain.copy()
    F = freq.copy()
    P = phase.copy()
    fig, axes = plt.subplots(2,1)
    fig.set_size_inches(8,8)
    axes[0].plot(F,G,"#ff0000")
    axes[1].plot(F,P,"#00ff00")
    axes[0].set_xscale('log')
    axes[1].set_xscale('log')
    axes[0].set_xlabel("freq (Hz)")
    axes[1].set_xlabel("freq (Hz)")
    axes[0].set_ylabel("Amp (dB)")
    axes[1].set_ylabel("phase (deg)")
    if freqHz!=0:
        axes[0].axvline(freqHz,color="b",linestyle="dotted")
        axes[1].axvline(freqHz,color="b",linestyle="dotted")
    axes[0].grid(True)
    axes[1].grid(True)

    plt.tight_layout()
    plt.show()
    
def makePhasePlotSim2(freq1, gain1, phase1,freq2, gain2, phase2):
  #  freq = processFile(filename, start_line,end_line, col=0)
  #  gain = processFile(filename, start_line,end_line, col=1)
  #  phase = processFile(filename, start_line,end_line, col=3,dbg=False)
    G1 = gain1.copy()*(-1.)
    G2 = gain2.copy()*(-1.)
    P1 = phase1.copy()
    P2 = phase2.copy()
    F1 = freq1.copy()
    F2 = freq2.copy()
    fig, axes = plt.subplots(2,1)
    fig.set_size_inches(8,8)
    axes[0].plot(F1,G1,"#ff0000",label="sim")
    axes[1].plot(F1,P1,"#ff0000",label="sim")
    axes[0].plot(F2,G2,"#00ff00",marker="o",label="data")
    axes[1].plot(F2,P2,"#00ff00",marker="o",label="data")
    axes[0].set_xscale('log')
    axes[1].set_xscale('log')
    axes[0].set_xlabel("freq (Hz)")
    axes[1].set_xlabel("freq (Hz)")
    axes[0].set_ylabel("Amp (dB)")
    axes[1].set_ylabel("phase (deg)")
    axes[0].grid(True)
    axes[1].grid(True)
    axes[1].legend(loc="lower right")
    axes[0].legend(loc="lower left")
    axes[1].grid(True)
    plt.tight_layout()
    plt.show()    
    

def makeOscPlot(t1, ch1, t2, ch2, offset = 0):
    fig, axes = plt.subplots(1,1)
    t1_copy = t1.copy() - offset
    t2_copy = t2.copy()-offset
    axes.plot(t1_copy,ch1,"#ff0000",label="ch1 (source)")
    axes.plot(t2_copy,ch2,"#0000ff",label="ch2 (response)")
    axes.set_xlabel("t (us)")
    axes.set_ylabel("U (V)")
    axes.legend(loc="lower right")
    plt.tight_layout()
    plt.show()

class osci_data:
    def __init__(self): 
        self.t1= None # czas 1 
        self.t2 = None
        self.ch1= None # kanal 1
        self.ch2 = None
    
class filter_dataset:
    def __init__(self):
        self.freq_data=None
        self.freq_sim=None
        self.phase_data=None
        self.phase_sim = None
        self.gain_data = None
        self.gain_sim = None
        self.freq_01_sim = osci_data()
        self.freq_10_sim = osci_data()
        self.freq_01_data = osci_data()
        self.freq_10_data = osci_data()

## Filtr dolnoprzepustowy:

Dane:

# <font color="red">Parsowanie do tablic:</font>
UWAGA:
genralnie to tylko przykład użycia kodu, w sprawozdaniu może zajść konieczność optymalizacji poniższych ustawień np.:
* w celu poprawnego opisu osi
* w celu wygenerowania ładnego wykresu (np. nie ma sensu generować wykresu gdzie przebiegi są "za gęste" i nic nie widać - można "uciąć" dane

In [ ]:

processFile("freq_sim","data/gor_bode.txt",4,14,0)
processFile("gain_sim","data/gor_bode.txt",4,14,1)
processFile("phase_sim","data/gor_bode.txt",4,14,2)

processFile("freq_data","data/gor_bode_real.txt",4,14,0)
processFile("gain_data","data/gor_bode_real.txt",4,14,1)
processFile("phase_data","data/gor_bode_real.txt",4,14,2)

nLines = 506
nStart = 6
nStep = 1
path = "data/100_hz_gor.txt"

processFile("t1_01_sim",path,nStart,nLines,1,magic_time=True, step=nStep)
processFile("t2_01_sim",path,nStart,nLines,4,magic_time=True, step=nStep)

processFile("ch1_01_sim",path,nStart,nLines,2, step=nStep)
processFile("ch2_01_sim",path,nStart,nLines,5, step=nStep)


nLines = 2506
nStep =100
path = "data/100_hz_gor_real.txt"
processFile("t1_01_data",path,nStart,nLines,1,magic_time=True, step=nStep)
processFile("t2_01_data",path,nStart,nLines,4,magic_time=True,step=nStep)

processFile("ch1_01_data",path,nStart,nLines,2,step=nStep)
processFile("ch2_01_data",path,nStart,nLines,5,step=nStep)


nLines = 506
nStep =50
path = "data/10k_hz_gor_real.txt"
processFile("t1_10_data",path,nStart,nLines,1,magic_time=True, step=nStep)
processFile("t2_10_data",path,nStart,nLines,4,magic_time=True,step=nStep)
processFile("ch1_10_data",path,nStart,nLines,2,step=nStep)
processFile("ch2_10_data",path,nStart,nLines,5,step=nStep)


nLines = 506

path = "data/10k_hz_gor.txt"

processFile("t1_10_sim",path,nStart,nLines,1,magic_time=True, step=nStep)
processFile("t2_10_sim",path,nStart,nLines,4,magic_time=True,step=nStep)
processFile("ch1_10_sim",path,nStart,nLines,2, step=nStep)
processFile("ch2_10_sim",path,nStart,nLines,5, step=nStep)



In [ ]:
# tutaj tworzymy wypełnione tablice
# w zasadzie przeklejając output z góry

In [ ]:
Filter =  filter_dataset()

# tutaj sobie tworzymy wypełnione tablice 

# freq - czestotliwosci
# data - dane
# sim - symulacja
# gain - amplitudoa
# phase - faza
#freq_01 dane dla f = 0.1 fg
#freq_10 dane dla f = 10 fg

Filter.freq_sim = freq_sim
Filter.gain_sim = gain_sim
Filter.phase_sim = phase_sim

Filter.gain_data = gain_data
Filter.freq_data = freq_data
Filter.phase_data = phase_data

Filter.freq_01_sim.t1 = t1_01_sim 
Filter.freq_01_sim.t2 = t2_01_sim
Filter.freq_01_sim.ch1 = ch1_01_sim
Filter.freq_01_sim.ch2 = ch2_01_sim
Filter.freq_01_data.t1 = t1_01_data
Filter.freq_01_data.t2 = t2_01_data
Filter.freq_01_data.ch1 = ch1_01_data 
Filter.freq_01_data.ch2 = ch2_01_data

Filter.freq_10_sim.t1 = t1_10_sim
Filter.freq_10_sim.t2 = t2_10_sim
Filter.freq_10_sim.ch1 = ch1_10_sim
Filter.freq_10_sim.ch2 = ch2_10_sim
Filter.freq_10_data.t1 = t1_10_data
Filter.freq_10_data.t2 = t2_10_data
Filter.freq_10_data.ch1 = ch1_10_data 
Filter.freq_10_data.ch2 = ch2_10_data

### Symulacja
Wykres charektrystyki amplitudowej i fazowej.

In [ ]:
makePhasePlotSim(Filter.freq_sim, Filter.gain_sim, Filter.phase_sim)


Odpowiedź układu na sygnał prostokątny o częstotliwośi 0.1 fg

In [ ]:
makeOscPlot(Filter.freq_01_sim.t1,Filter.freq_01_sim.ch1,Filter.freq_01_sim.t2,Filter.freq_01_sim.ch2 ,offset=740000)

Odpowiedź układu na sygnał prostokątny o częstotliwośi 10 fg

In [ ]:
makeOscPlot(Filter.freq_10_sim.t1,Filter.freq_10_sim.ch1,Filter.freq_10_sim.t2,Filter.freq_10_sim.ch2 ,offset=740000)

### Pomiar
Wykres charakterystyki fazowej i amplitudowej.

In [ ]:
makePhasePlotSim(Filter.freq_data, Filter.gain_data, Filter.phase_data)

Odpowiedź układu na sygnał prostokątny o częstotliwośi 0.1 fg

In [ ]:
makeOscPlot(Filter.freq_01_data.t1,Filter.freq_01_data.ch1,Filter.freq_01_data.t2,Filter.freq_01_data.ch2 )

Odpowiedź układu na sygnał prostokątny o częstotliwośi 10 fg

In [ ]:
makeOscPlot(Filter.freq_10_data.t1,Filter.freq_10_data.ch1,Filter.freq_10_data.t2,Filter.freq_10_data.ch2 )